Continuing work from preprocessing1a, just because that notebook was running too slowly: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing1a_choose_users.ipynb

Previously, I selected a random subset of users and created rows for non-orders. Here, I'll need to start by cleaning up all those rows before building the 'past_orders' column. 

In [1]:
import pandas as pd
import numpy as np
import os
from library.sb_utils import save_file

import random

In [2]:
from IPython.display import Audio
sound_file = './alert.wav'

In [3]:
# import users_incorrect_deets from previous notebook
df = pd.read_csv('../data/processed/users_incorrect_deets.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4254 entries, 0 to 4253
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                4254 non-null   int64  
 1   user_id                 4254 non-null   int64  
 2   order_by_user_sequence  4254 non-null   int64  
 3   order_dow               4254 non-null   int64  
 4   order_hour_of_day       4254 non-null   int64  
 5   days_since_prior_order  4254 non-null   float64
 6   add_to_cart_sequence    4254 non-null   int64  
 7   reordered               4254 non-null   int64  
 8   product_name            4254 non-null   object 
 9   aisle_name              4254 non-null   object 
 10  dept_name               4254 non-null   object 
dtypes: float64(1), int64(7), object(3)
memory usage: 365.7+ KB


In [4]:
# This actually needs to be changed based on how I re-did adding of rows!!!!!!!!!!!!!!!!!!!

# The new rows were created without updating orde_id, order_dow, order_hour_of_day, 
# days_since_prior_order. Fix this.

users = list(df['user_id'].unique())
sdf
    this_users_rows = df.loc[df['user_id']==user, :].copy()
    
    # Iterate over each order for this user
    for order in range(2,(
        df[df['user_id']==user]['order_by_user_sequence'].max()+1)):
        
        this_order = this_users_rows.loc[this_users_rows[
            'order_by_user_sequence']==order, :].copy()
        
        # Get the values columns should have for this order
        # Correct values are where add_to_cart_sequence>0
        try: 
            correct_order_id = this_order[this_order[
                'add_to_cart_sequence']>0]['order_id'].values[0]
            correct_order_dow = this_order[this_order[
            'add_to_cart_sequence']>0]['order_dow'].values[0]
            correct_order_hour_of_day = this_order[this_order[
                'add_to_cart_sequence']>0]['order_hour_of_day'
                                          ].values[0]
            correct_days_since_prior_order = this_order[this_order[
                'add_to_cart_sequence']>0]['days_since_prior_order'
                                          ].values[0]
        except IndexError:
            pass
        
        # Change values in original df, just rows for this order
        # So that all rows have the correct deets
        df.loc[(df['user_id']==user)&(df[
            'order_by_user_sequence']==order),'order_id'
              ] = correct_order_id
        df.loc[(df['user_id']==user)&(df[
            'order_by_user_sequence']==order),'order_dow'
              ] = correct_order_dow
        df.loc[(df['user_id']==user)&(df[
            'order_by_user_sequence']==order),'order_hour_of_day'
              ] = correct_order_hour_of_day
        df.loc[(df['user_id']==user)&(df[
            'order_by_user_sequence']==order),
               'days_since_prior_order'
                        ] = correct_days_since_prior_order

In [5]:
# Check to see if the change was made correctly.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4254 entries, 0 to 4253
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                4254 non-null   int64  
 1   user_id                 4254 non-null   int64  
 2   order_by_user_sequence  4254 non-null   int64  
 3   order_dow               4254 non-null   int64  
 4   order_hour_of_day       4254 non-null   int64  
 5   days_since_prior_order  4254 non-null   float64
 6   add_to_cart_sequence    4254 non-null   int64  
 7   reordered               4254 non-null   int64  
 8   product_name            4254 non-null   object 
 9   aisle_name              4254 non-null   object 
 10  dept_name               4254 non-null   object 
dtypes: float64(1), int64(7), object(3)
memory usage: 365.7+ KB


In [6]:
df['user_id'].unique()

array([ 10871,  36717,  52948,  68945,  77355,  82400,  88752,  91835,
        92521,  94303,  96871, 100293, 102226, 107632, 112562, 122429,
       140900, 141897, 179747, 188389])

In [7]:
df[(df['user_id']==10871) & df['order_by_user_sequence'].isin([
    2,3])].sort_values('order_by_user_sequence')

,order_id,user_id,order_by_user_sequence,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_sequence,reordered,product_name,aisle_name,dept_name
9,754784,10871,2,0,9,30.0,2,0,Carrots,fresh vegetables,produce
16,754784,10871,2,0,9,30.0,6,0,Pretzel Crisps Original Deli Style Pretzel Cra...,chips pretzels,snacks
15,754784,10871,2,0,9,30.0,1,1,Fat Free Milk,milk,dairy eggs
14,754784,10871,2,0,9,30.0,4,0,Original Hummus,fresh dips tapenades,deli
17,754784,10871,2,0,9,30.0,5,0,Organic Hothouse Cucumbers,fresh vegetables,produce
12,754784,10871,2,0,9,30.0,3,0,Red Peppers,fresh vegetables,produce
11,754784,10871,2,0,9,30.0,9,0,Organic Roasted Turkey Breast,lunch meat,deli
10,754784,10871,2,0,9,30.0,8,0,Organic Strawberries,fresh fruits,produce
13,754784,10871,2,0,9,30.0,7,0,Sea Salt Pita Chips,chips pretzels,snacks
25,1256881,10871,3,5,16,30.0,8,0,Honey Whole Wheat Bread,bread,bakery


Each order now has the same order details (order_id, day, hour, days_since_prior) in each row of the order, correctly matching items actually purchased in that order.

Now I need to use the dictionaries I created earlier to fill in the aisle_name and dept_name based on the product_name. 

Up next is preparing to create a column that indicates how many times any given item has been ordered in the past. 

My intuition is that cross-row calculations don't count as leakage and negatively impact modeling IF I'm adding data that has exclusively to do with past orders. If this logic turns out to be inappropriate, I can just come back and split the set into separate users (or into certain orders per user) and re-run any subsequent code. 

In [10]:
# I might want a dictionary of which products were reordered in each order, by each user. 

prod_reorder_by_user_per_order = {}

for user in users:
    products_reordered_each_order = {}
    for order in range(2,101):
        items = set(df[(df['user_id']==user) & (df[
            'order_by_user_sequence']==order) & (df[
            'reordered']==1)]['product_name'])
        if len(items)>0:
            products_reordered_each_order[order]=items
    prod_reorder_by_user_per_order[user]=products_reordered_each_order
    
prod_reorder_by_user_per_order

{10871: {2: {'Fat Free Milk'},
  3: {'Fat Free Milk',
   'Organic Hothouse Cucumbers',
   'Organic Roasted Turkey Breast',
   'Original Hummus',
   'Red Peppers'},
  4: {'Honey Whole Wheat Bread',
   'Organic Hass Avocado',
   'Organic Hothouse Cucumbers',
   'Organic Strawberries'},
  5: {'Sonoma Traditional Flour Tortillas 10 Count'},
  6: {'Bag of Organic Bananas',
   'Blackberry Yogurt',
   'Organic Fat Free Milk',
   'Organic Strawberries',
   'Total 0% Raspberry Yogurt',
   'Total 2% Greek Strained Yogurt with Cherry 5.3 oz',
   'Total 2% Lowfat Greek Strained Yogurt with Peach',
   'Total 2% with Strawberry Lowfat Greek Strained Yogurt'},
  7: {'Sea Salt Caramel Ice Cream',
   'Stringless Sugar Snap Peas',
   'Total 2% Lowfat Greek Strained Yogurt with Peach'},
  8: {'Bag of Organic Bananas',
   'Blackberry Yogurt',
   'Honey Whole Wheat Bread',
   'Organic Fat Free Milk',
   'Organic Gala Apples',
   'Organic Hass Avocado',
   'Organic Hothouse Cucumbers',
   'Organic Roasted T

Create the column itself, starting as just a copy of 'reordered'. 

For each user's first order, this column will already be true because reordered=0 and past_orders=0 since there were no past orders. For their second order, it will be partly true. For order 2, if rordered=1, then it's because the item was ordered once in the past (in the first order). For order 2, if add_to_cart_sequence=0, the past_order value should actually be 1 because that indicates non-orders for order 2 of items previously ordered. The only items in order 2 that should have a past_order value of 0 should be items being ordered for the first time in order 2. 

In [12]:
df['past_orders'] = df.loc[:,'reordered'].copy()

In [13]:
# To start with, just work with 2 users, to more quickly see if the code works

all_users = set(df['user_id'].unique())
practice_users = random.sample(list(all_users), 2)
df0 = df.loc[df['user_id'].isin(practice_users), :].copy()
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 396 to 4249
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   order_id                351 non-null    int64  
 1   user_id                 351 non-null    int64  
 2   order_by_user_sequence  351 non-null    int64  
 3   order_dow               351 non-null    int64  
 4   order_hour_of_day       351 non-null    int64  
 5   days_since_prior_order  351 non-null    float64
 6   add_to_cart_sequence    351 non-null    int64  
 7   reordered               351 non-null    int64  
 8   product_name            351 non-null    object 
 9   aisle_name              351 non-null    object 
 10  dept_name               351 non-null    object 
 11  reorders_so_far         351 non-null    int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 35.6+ KB


In [ ]:
# Do the work here

In [21]:
# Check to see if this worked. 

KeyError: 'actual_reorders_so_far'

In [ ]:
# Engineer columns for product keywords

In [ ]:
# Change format of dow, hour columns

In [ ]:
# Can't do the rest of the normalization until I have a train/test split and pipeline for it
# Save work done so far as new csv file and pick up in next notebook

Move to new notebook for encoding & standardizing remaining features